# Transformer recipe generator training

Note: This notebook is designed to run with GPU runtime.

Install Huggingface libraries to use the pretrained tokenizer and the recipe dataset.

**You can igore the error message like `ERROR: pip's dependency resolver does not currently take into account...`.**

In [1]:
pip install -q git+https://github.com/huggingface/transformers.git datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_mac

Import modules and set random seeds.

In [2]:
import os, random
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, saving

random.seed(20230629)
np.random.seed(20230629)
tf.random.set_seed(20230629)

plt.rcParams.update({'font.size': 10})

Download the pretrained tokenizer and check the vacabulary size.

In [3]:
from transformers import AutoTokenizer, AutoConfig
model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
VOCAB_SIZE = AutoConfig.from_pretrained(model_ckpt).vocab_size

print(f'Vocabulary size: {VOCAB_SIZE}')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Vocabulary size: 30522


Download the recipe dataset and extract directions texts.

In [4]:
from datasets import load_dataset
recipe = load_dataset('Shengtao/recipe')

def join_title_and_directions(title_directions):
    title, directions = title_directions
    return f'Recipe for {title}: {directions}'

recipe_texts = zip(recipe['train']['title'], recipe['train']['directions'])
recipe_texts = [*map(join_title_and_directions, recipe_texts)]
recipe_texts = recipe_texts[::4] # Select 25% of the entire training set.

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

recipe.csv:   0%|          | 0.00/64.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32722 [00:00<?, ? examples/s]

Create the training and test datasets, truncating long texts into 512 words.

In [5]:
MAX_LEN = 128
train_set, test_set = train_test_split(recipe_texts, test_size=0.1)

train_set = tokenizer(train_set, max_length=MAX_LEN,
                      padding='max_length', truncation=True)
train_text = np.array(train_set['input_ids'])[:, :-1]
train_label = np.array(train_set['input_ids'])[:, 1:]

test_set = tokenizer(test_set, max_length=MAX_LEN,
                     padding='max_length', truncation=True)
test_text = np.array(test_set['input_ids'])[:, :-1]
test_label = np.array(test_set['input_ids'])[:, 1:]

Define the positional embedding layer.

In [6]:
@saving.register_keras_serializable()
class Embeddings(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim, **kwargs):
        super(Embeddings, self).__init__(**kwargs)
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size,
                                          output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=max_len,
                                        output_dim=embed_dim)
        self.dropout = layers.Dropout(rate=0.5)
        self.ln = layers.LayerNormalization(epsilon=1e-12)

    def build(self, input_shape):
        pass

    def call(self, inputs):
        seq_len = tf.shape(inputs)[-1]
        position_ids = tf.range(start=0, limit=seq_len, delta=1)
        position_embeddings = self.pos_emb(position_ids)
        token_embeddings = self.token_emb(inputs)

        # Add positional embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.ln(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

Dfiner the Transformer encoder block.

In [7]:
@saving.register_keras_serializable()
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(rate=0.1)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation='relu')
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(rate=0.1)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def build(self, input_shape):
        pass

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        seq_len = tf.shape(inputs)[1]
        causal_mask = self.causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool)

        # Multi-head attention
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        attention_output =  attention_output + inputs # Skip connection
        attention_output = self.ln_1(attention_output)

        # Feed forward
        ffn_1 = self.ffn_1(attention_output)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        ffn_output = ffn_output + attention_output # Skip connection
        ffn_output = self.ln_2(ffn_output)
        return (ffn_output, attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
            }
        )
        return config

Define the Transformer model consisted of a single Transformer encoder blocks.

In [8]:
EMBEDDING_DIM = 512
N_HEADS = 4 # 8
KEY_DIM = EMBEDDING_DIM // N_HEADS
FEED_FORWARD_DIM = 2048

text_inputs = layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')
x = Embeddings(
    MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM, name='text_embedding')(text_inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM, name='Transformer')(x)
output = layers.Dense(VOCAB_SIZE, activation='softmax', name='softmax')(x)

# Model for training and prediction
transformer_model = models.Model(
    inputs=text_inputs, outputs=output,
    name='Transformer_next_word_predictor')

# Model for inference including attention scores
attention_scores_model = models.Model(
    inputs=text_inputs,
    outputs=[output, attention_scores],
    name='Transformer_attention_scores')

transformer_model.summary()

Model: "Transformer_next_word_predictor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)               │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_embedding (Embeddings)          │ (None, None, 512)           │      15,693,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Transformer (TransformerBlock)       │ [(None, None, 512), (None,  │       3,152,384 │
│                                      │ 4, None, None)]             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Dense)                      │ (None, None, 30522)         │      15,657,786 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,503,994 (131.62 MB)

 Trainable params: 34,503,994 (131.62 MB)

 Non-trainable params: 0 (0.00 B)

Compile the model using the Adam optimizer, and the sparse categorical crossentroy as a loss function.

In [9]:
transformer_model.compile('adam',
                          loss='sparse_categorical_crossentropy',
                          metrics=['acc'])

Train the model.

In [10]:
%%time
history = transformer_model.fit(train_text, train_label,
                                validation_data=(test_text, test_label),
                                batch_size=8, epochs=10)

Epoch 1/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 61s 54ms/step - acc: 0.3967 - loss: 3.5737 - val_acc: 0.5348 - val_loss: 2.2779
Epoch 2/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 70s 49ms/step - acc: 0.5318 - loss: 2.2350 - val_acc: 0.5707 - val_loss: 2.0572
Epoch 3/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 83s 50ms/step - acc: 0.5627 - loss: 2.0047 - val_acc: 0.5858 - val_loss: 1.9576
Epoch 4/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 81s 48ms/step - acc: 0.5795 - loss: 1.8743 - val_acc: 0.5955 - val_loss: 1.9051
Epoch 5/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 83s 49ms/step - acc: 0.5921 - loss: 1.7838 - val_acc: 0.6009 - val_loss: 1.8787
Epoch 6/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 45s 49ms/step - acc: 0.6007 - loss: 1.7160 - val_acc: 0.6041 - val_loss: 1.8550
Epoch 7/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 82s 48ms/step - acc: 0.6084 - loss: 1.6626 - val_acc: 0.6073 - val_loss: 1.8436
Epoch 8/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 83s 49ms/step - acc: 0.6141 - loss: 1.6168 - val_acc: 0.6104 - val_loss: 1.8303
Epoch 9/10
921/921 ━━━━━━━━━━━━━━━━━━━━ 

Mount the google drive and save the trained model.

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

model_file = '/content/gdrive/My Drive/Transformer_recipe_generator.keras'
transformer_model.save(model_file)
!ls -lh "{model_file}"

model_file = '/content/gdrive/My Drive/Transformer_attention_scores.keras'
attention_scores_model.save(model_file)
!ls -lh "{model_file}"

Mounted at /content/gdrive
-rw------- 1 root root 395M Jan 30 06:58 '/content/gdrive/My Drive/Transformer_recipe_generator.keras'
-rw------- 1 root root 132M Jan 30 06:58 '/content/gdrive/My Drive/Transformer_attention_scores.keras'
